In [48]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [49]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print(intents)

{u'intents': [{u'patterns': [u'Tere', u'Hei', u'Tsau', u'Tsauki'], u'tag': u'name', u'responses': [u'Tere, mis su nimi on? \U0001f600\U0001f600\U0001f600'], u'context_set': u'name'}, {u'responses': [u'Ole n\xfc\xfcd, saame tuttavaks, vana sa oled? \U0001f600', u'\xc4ra karda, ma tunnen su vanemaid, kui vana sa oled? \U0001f600'], u'positive': False, u'patterns': [u'Ei \xfctle', u'Ei'], u'tag': u'age_neg', u'context_filter': u'name', u'context_set': u'age'}, {u'responses': [u'R\xf5\xf5m tutvuda, ma olen Mari ja olen 14-aastane. Kui vana sa oled?', u'\U0001f600 Tore! Ma olen Martin, 13. Vana sa?', u'Ma olen Kadri, kohe saan 12. Kui vana sa praegu oled?'], u'positive': True, u'patterns': [u'Olen', u'Minu nimi', u' '], u'tag': u'age_pos', u'context_filter': u'name', u'context_set': u'age'}, {u'responses': [u'Pole hullu, k\xfcll sa varsti usaldad mind! Mul on kodus koer ja me elame perega Viimsis suures majas. Kus sa elad?', u'Vanus on k\xf5igest number \U0001f600. Mul on kodus kass ja ma e

In [50]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(100, 'documents')
(18, 'classes', [u'age_neg', u'age_pos', u'facebook_neg', u'facebook_pos', u'goodbye', u'grade_neg', u'grade_pos', u'home_neg', u'home_pos', u'location_neg', u'location_pos', u'name', u'pets_neg', u'pets_pos', u'subscribe_neg', u'subscribe_pos', u'youtube_neg', u'youtube_pos'])
(62, 'unique stemmed words', [u'!', u'1', u'10', u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'2', u'3', u'4', u'5', u'6', u'7', u'8', u'9', u'aast', u'aeg', u'as', u'bak', u'eelkool', u'ei', u'el', u'hamst', u'head', u'hei', u'hiir', u'ikk', u'ja', u'jah', u'jh', u'kass', u'koer', u'kort', u'k\xfcll', u'lastea', u'maja', u'merisig', u'minu', u'mkm', u'muj', u'nim', u'n\xe4gemist', u'ol', u'on', u'poe', u'pol', u'see', u'siin', u'sinu', u's\xf5bra', u'tahak', u'ter', u'tsau', u'tsauk', u't\xf6\xf6l', u'vaj', u'van', u'\xfclikool', u'\xfctle'])


In [51]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [52]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 12999  | total loss: 0.82870 | time: 0.037s
| Adam | epoch: 1000 | loss: 0.82870 - acc: 0.6062 -- iter: 096/100
Training Step: 13000  | total loss: 0.82756 | time: 0.040s
| Adam | epoch: 1000 | loss: 0.82756 - acc: 0.6206 -- iter: 100/100
--
INFO:tensorflow:/home/salts/cybersecurity-exhibition/chatbot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [53]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [54]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[u'age_neg', u'age_pos', u'facebook_neg', u'facebook_pos', u'goodbye', u'grade_neg', u'grade_pos', u'home_neg', u'home_pos', u'location_neg', u'location_pos', u'name', u'pets_neg', u'pets_pos', u'subscribe_neg', u'subscribe_pos', u'youtube_neg', u'youtube_pos']


In [55]:
print(model.predict([p]))

[[1.2318924e-05 1.3873255e-01 8.1763286e-03 1.7084652e-01 2.5544237e-08
  2.2495251e-04 1.4882818e-02 4.9710219e-05 9.1532141e-02 5.5388999e-05
  1.8111272e-05 6.1881146e-04 1.0003788e-05 1.0571235e-01 2.3274424e-06
  2.8498334e-01 8.1595788e-03 1.7598271e-01]]


In [56]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )